## Does Compulsory School Attendance Affect Schooling and Earnings?

## 背景故事

Angrist和Krueger(1991)的出生季度识别策略的出发点，是观察到大多数州要求学生在他们满6岁的公历年入学。因此，入学年龄是出生日期的函数。具体来说，年尾出生的孩子在他们的年级里比较小。在将12月31日设定为生日截止日期的州，第四季度出生的孩子在不满6岁时就已入学，而第一季度出生的孩子在6岁半左右才入学。此外，由于义务教育法通常要求学生必须在学校待到16岁生日，所以当这些学生群体达到法定的辍当学生达到入学年龄时，他们可能会处于不同的年级，或者处于同一年级但上学的时间长短不同。学校的入学年龄政策和义务教育法的结合创造了一个自然实验，在这个实验中，孩子们被迫根据他们的生日去接受不同时长的学校教育。

Angrist 和 Krueger (1991)
利用美国人口普查数据研究了受教育程度和出生季度之间的关系。图A \[改编自
Angrist 和 Krueger (1991)\] 显示了 1980 年人口普查记录中出生在 20 世纪
30
年代不同出生季度下的男性的受教育情况。这一数据清楚地表明，在公历年中出生较早的男性，其平均受教育程度往往较低。图A
是第一阶段的图形描述。一般来说，工具变量框架中的第一阶段是感兴趣的因果关系变量对协变量和工具变量的回归。这张图总结了这一回归，因为按出生年份和出生季度计算的平均受教育程度是对出生年份虚拟变量（协变量）和出生季度虚拟变量（工具变量）进行受教育程度回归后得到的拟合值。

图B 显示了用于构造图A 的相同样本按出生季度给出的平均收入。图B
说明了工具变量和因变量之间简化形式下的关系。简化形式是因变量对模型中的协变量和工具变量进行的回归。图B
显示，年长组往往收入更高，因为收入会随着工作经验增加而增加。

该数据还表明，出生在较早季度的男性的平均收入几乎总是低于出生在较晚季度的男性，即使在调整出生年份\[Angrist
和
Krueger(1991)架构中的协变量\]后也是如此。重要的是，这种简化形式的关系与学校教育中出生季度的模式相似，表明这两种模式密切相关。因为一个人的出生日期很可能与其先天能力、动机或家庭关系无关，所以似乎可以可信地断言，收入随出生季度上下波动的唯一原因是受教育程度随出生季度上下波动。这是推动出生季度这一工具变量产生的关键假设。

## 初始化

``` stata
* 清理工作空间并关闭分页显示
clear all
set more off

* 导入数据文件
* 变量说明：
* lwklywge: 周工资对数
* educ: 受教育年限
* yob: 出生年份
* qob: 出生季度
* pob: 出生地
infile lwklywge educ yob qob pob using data/asciiqob.txt, clear
```

    (329,509 observations read)

绘制图形

``` stata
* 生成季度日期变量
* 将出生年份转换为1900年后的年份，并与季度组合
gen yqob = yq(1900 + yob, qob)
format yqob %tq  // 设置日期格式为季度格式

* 按出生季度汇总数据
* 计算每个季度的平均受教育年限、周工资对数和季度值
collapse (mean) educ (mean) lwklywge (mean) qob, by(yqob)

* 绘制教育年限与出生季度的关系图
* 第一部分：教育年限与出生季度的关系（第一阶段）
graph twoway (line educ yqob, lcolor(black))                                        ///
             (scatter educ yqob if qob == 1,                                        ///
                mlabel(qob) msize(small) msymbol(S) mcolor(black))                  ///
             (scatter educ yqob if qob != 1,                                        ///
                mlabel(qob) msize(small) msymbol(Sh) mcolor(black)),                ///
                    xlabel(, format(%tqY))                                          ///
                    title("A. 出生季度与平均受教育年限的关系（第一阶段）")         ///
                    ytitle("受教育年限（年）")                                      ///
                    xtitle("出生年份")                                              ///
                    legend(off)                                                     ///
                    name(educ)                                                      ///
                    scheme(s1mono)

* 第二部分：周工资对数与出生季度的关系（简化形式）
graph twoway (line lwklywge yqob, lcolor(black))                                   ///
             (scatter lwklywge yqob if qob == 1,                                   ///
                mlabel(qob) msize(small) msymbol(S) mcolor(black))                 ///
             (scatter lwklywge yqob if qob != 1,                                   ///
                mlabel(qob) msize(small) msymbol(Sh) mcolor(black)),               ///
                    xlabel(, format(%tqY))                                         ///
                    title("B. 出生季度与平均周工资的关系（简化形式）")                ///
                    ytitle("周工资对数")                                           ///
                    xtitle("出生年份")                                             ///
                    legend(off)                                                    ///
                    name(lwklywge)                                                 ///
                    scheme(s1mono)
```

## 回归分析

``` stata
* ------------------------------------------------------------------------------
* 回归分析
* ------------------------------------------------------------------------------
infile lwklywge educ yob qob pob using data/asciiqob.txt, clear

* 1 基准OLS回归
regress lwklywge educ, robust
```

    (329,509 observations read)


    Linear regression                               Number of obs     =    329,509
                                                    F(1, 329507)      =   34577.15
                                                    Prob > F          =     0.0000
                                                    R-squared         =     0.1173
                                                    Root MSE          =     .63777

    ------------------------------------------------------------------------------
                 |               Robust
        lwklywge |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
    -------------+----------------------------------------------------------------
            educ |    .070851    .000381   185.95   0.000     .0701042    .0715978
           _cons |   4.995182   .0050739   984.49   0.000     4.985238    5.005127
    ------------------------------------------------------------------------------

``` stata
* 2 加入出生年份和出生地虚拟变量的OLS回归
regress lwklywge educ i.yob i.pob, robust
```

    Linear regression                               Number of obs     =    329,509
                                                    F(60, 329448)     =     649.29
                                                    Prob > F          =     0.0000
                                                    R-squared         =     0.1288
                                                    Root MSE          =     .63366

    ------------------------------------------------------------------------------
                 |               Robust
        lwklywge |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
    -------------+----------------------------------------------------------------
            educ |    .067339   .0003883   173.40   0.000     .0665778    .0681001
                 |
             yob |
             31  |  -.0052304   .0050958    -1.03   0.305    -.0152179    .0047571
             32  |  -.0106598   .0050202    -2.12   0.034    -.0204992   -.0008204
             33  |  -.0125521   .0050378    -2.49   0.013     -.022426   -.0026782
             34  |  -.0160905   .0050341    -3.20   0.001    -.0259573   -.0062238
             35  |  -.0276552   .0050114    -5.52   0.000    -.0374774   -.0178329
             36  |   -.027147   .0049417    -5.49   0.000    -.0368326   -.0174614
             37  |  -.0323545   .0048659    -6.65   0.000    -.0418916   -.0228173
             38  |  -.0331394   .0048341    -6.86   0.000    -.0426141   -.0236647
             39  |   -.043888   .0048085    -9.13   0.000    -.0533126   -.0344634
                 |
             pob |
              2  |   .2588778   .0949633     2.73   0.006     .0727526    .4450031
              4  |   .1464813   .0193755     7.56   0.000     .1085059    .1844567
              5  |   .0457005   .0116372     3.93   0.000      .022892    .0685091
              6  |   .1736215   .0097964    17.72   0.000     .1544207    .1928222
              8  |   .1209974   .0141371     8.56   0.000      .093289    .1487058
              9  |   .1340424   .0123279    10.87   0.000       .10988    .1582047
             10  |   .0899371   .0240266     3.74   0.000     .0428457    .1370285
             11  |   .1540621   .0188787     8.16   0.000     .1170604    .1910637
             12  |   .0016072   .0128701     0.12   0.901    -.0236178    .0268323
             13  |  -.0192893   .0106452    -1.81   0.070    -.0401536     .001575
             15  |    .152859   .0421279     3.63   0.000     .0702894    .2354286
             16  |   .1046053   .0174079     6.01   0.000     .0704864    .1387243
             17  |   .2057122   .0090057    22.84   0.000     .1880613     .223363
             18  |   .1531159   .0100317    15.26   0.000     .1334541    .1727777
             19  |   .0991303   .0112729     8.79   0.000     .0770357    .1212249
             20  |   .0734366   .0120887     6.07   0.000      .049743    .0971302
             21  |    .111496   .0104622    10.66   0.000     .0909904    .1320017
             22  |   .0948181   .0116703     8.12   0.000     .0719447    .1176915
             23  |  -.0119525   .0142945    -0.84   0.403    -.0399693    .0160643
             24  |   .1179354   .0125133     9.42   0.000     .0934097    .1424612
             25  |   .1101806   .0097968    11.25   0.000     .0909791    .1293821
             26  |   .2266721    .009239    24.53   0.000     .2085638    .2447804
             27  |    .150167   .0108328    13.86   0.000      .128935    .1713991
             28  |  -.0210854   .0119568    -1.76   0.078    -.0445203    .0023495
             29  |   .1133091   .0103281    10.97   0.000     .0930665    .1335518
             30  |   .0778245   .0198774     3.92   0.000     .0388654    .1167837
             31  |   .0905794   .0136966     6.61   0.000     .0637345    .1174244
             32  |   .1534007   .0410659     3.74   0.000     .0729127    .2338886
             33  |   .0195914   .0185698     1.06   0.291    -.0168049    .0559876
             34  |   .1700712   .0101306    16.79   0.000     .1502155    .1899269
             35  |    .070872    .017677     4.01   0.000     .0362256    .1055185
             36  |   .1613177   .0085519    18.86   0.000     .1445562    .1780792
             37  |  -.0564163   .0099888    -5.65   0.000     -.075994   -.0368386
             38  |   .1298529   .0167416     7.76   0.000       .09704    .1626659
             39  |   .1651131   .0089859    18.37   0.000      .147501    .1827252
             40  |   .0912515   .0108646     8.40   0.000     .0699571    .1125458
             41  |   .1414549   .0152077     9.30   0.000     .1116483    .1712615
             42  |     .13079   .0085238    15.34   0.000     .1140835    .1474964
             44  |   .0454543   .0169712     2.68   0.007     .0121912    .0787174
             45  |  -.0798036   .0123946    -6.44   0.000    -.1040966   -.0555106
             46  |   .0767939   .0170484     4.50   0.000     .0433796    .1102083
             47  |   .0482311   .0105788     4.56   0.000      .027497    .0689652
             48  |   .0879824   .0093379     9.42   0.000     .0696803    .1062844
             49  |   .1403073   .0152453     9.20   0.000     .1104269    .1701877
             50  |  -.0381869   .0211139    -1.81   0.071    -.0795696    .0031958
             51  |   .0319901    .010963     2.92   0.004     .0105029    .0534774
             53  |   .1882046   .0129013    14.59   0.000     .1629183    .2134908
             54  |   .1274668   .0108364    11.76   0.000     .1062278    .1487058
             55  |   .1276702   .0103486    12.34   0.000     .1073872    .1479531
             56  |   .1439475   .0260318     5.53   0.000      .092926     .194969
                 |
           _cons |   4.950867   .0095697   517.35   0.000     4.932111    4.969623
    ------------------------------------------------------------------------------

``` stata
* 3 使用第一季度作为工具变量的2SLS回归
tabulate qob, gen(qob)
ivregress 2sls lwklywge (educ = qob1), robust
```

            qob |      Freq.     Percent        Cum.
    ------------+-----------------------------------
              1 |     81,671       24.79       24.79
              2 |     80,138       24.32       49.11
              3 |     86,856       26.36       75.47
              4 |     80,844       24.53      100.00
    ------------+-----------------------------------
          Total |    329,509      100.00


    Instrumental variables (2SLS) regression          Number of obs   =    329,509
                                                      Wald chi2(1)    =      18.01
                                                      Prob > chi2     =     0.0000
                                                      R-squared       =     0.0946
                                                      Root MSE        =     .64591

    ------------------------------------------------------------------------------
                 |               Robust
        lwklywge |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
    -------------+----------------------------------------------------------------
            educ |    .101995   .0240315     4.24   0.000     .0548941    .1490959
           _cons |   4.597477    .306889    14.98   0.000     3.995985    5.198968
    ------------------------------------------------------------------------------
    Instrumented:  educ
    Instruments:   qob1

``` stata
* 4 加入出生年份和出生地虚拟变量的2SLS回归
ivregress 2sls lwklywge i.yob i.pob (educ = qob1), robust
```

    Instrumental variables (2SLS) regression          Number of obs   =    329,509
                                                      Wald chi2(60)   =   10050.32
                                                      Prob > chi2     =     0.0000
                                                      R-squared       =     0.0989
                                                      Root MSE        =      .6444

    ------------------------------------------------------------------------------
                 |               Robust
        lwklywge |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
    -------------+----------------------------------------------------------------
            educ |   .1041942   .0256662     4.06   0.000     .0538893     .154499
                 |
             yob |
             31  |  -.0105046   .0063456    -1.66   0.098    -.0229418    .0019325
             32  |  -.0184508   .0074405    -2.48   0.013    -.0330339   -.0038677
             33  |  -.0225605   .0086912    -2.60   0.009     -.039595    -.005526
             34  |  -.0275482   .0094856    -2.90   0.004    -.0461397   -.0089568
             35  |  -.0414848   .0109079    -3.80   0.000    -.0628639   -.0201057
             36  |  -.0426426   .0119135    -3.58   0.000    -.0659926   -.0192925
             37  |  -.0501027    .013324    -3.76   0.000    -.0762174   -.0239881
             38  |  -.0538935   .0152651    -3.53   0.000    -.0838125   -.0239746
             39  |  -.0660917   .0162526    -4.07   0.000    -.0979463   -.0342371
                 |
             pob |
              2  |   .1939485   .1076039     1.80   0.071    -.0169512    .4048481
              4  |   .0953137    .040712     2.34   0.019     .0155197    .1751078
              5  |   .0402742   .0123936     3.25   0.001     .0159833    .0645652
              6  |   .0946279   .0559251     1.69   0.091    -.0149833     .204239
              8  |   .0615939   .0438277     1.41   0.160    -.0243068    .1474945
              9  |   .0746528   .0431713     1.73   0.084    -.0099614    .1592671
             10  |   .0687893   .0282448     2.44   0.015     .0134304    .1241482
             11  |   .0763058   .0575112     1.33   0.185    -.0364142    .1890257
             12  |   -.033895   .0280365    -1.21   0.227    -.0888455    .0210556
             13  |  -.0115358   .0120107    -0.96   0.337    -.0350763    .0120048
             15  |   .0969065   .0581212     1.67   0.095    -.0170091     .210822
             16  |    .037387   .0500464     0.75   0.455    -.0607021     .135476
             17  |   .1451677   .0431796     3.36   0.001     .0605372    .2297983
             18  |   .1140211   .0290903     3.92   0.000     .0570052    .1710369
             19  |   .0464911   .0385013     1.21   0.227    -.0289701    .1219522
             20  |   .0089253   .0465974     0.19   0.848    -.0824039    .1002544
             21  |   .1277416    .015434     8.28   0.000     .0974915    .1579918
             22  |   .0814181   .0151509     5.37   0.000     .0517228    .1111134
             23  |  -.0358942   .0221186    -1.62   0.105    -.0792459    .0074575
             24  |   .0910844   .0226184     4.03   0.000     .0467531    .1354157
             25  |   .0448426   .0465359     0.96   0.335    -.0463661    .1360513
             26  |    .179116   .0344731     5.20   0.000     .1115499    .2466821
             27  |   .0956717   .0396337     2.41   0.016     .0179911    .1733524
             28  |  -.0127679   .0133751    -0.95   0.340    -.0389825    .0134468
             29  |   .0767956   .0274913     2.79   0.005     .0229136    .1306777
             30  |   .0164883   .0472727     0.35   0.727    -.0761644    .1091411
             31  |   .0298338   .0444934     0.67   0.503    -.0573716    .1170392
             32  |   .0889784   .0611492     1.46   0.146    -.0308719    .2088287
             33  |  -.0132188   .0294754    -0.45   0.654    -.0709896     .044552
             34  |   .1062962   .0456104     2.33   0.020     .0169015    .1956909
             35  |   .0386989   .0289424     1.34   0.181    -.0180271    .0954249
             36  |   .0876247   .0520388     1.68   0.092    -.0143695     .189619
             37  |  -.0559476    .010095    -5.54   0.000    -.0757334   -.0361619
             38  |   .0840985   .0361947     2.32   0.020     .0131582    .1550388
             39  |   .1194614    .033026     3.62   0.000     .0547317    .1841912
             40  |   .0433506   .0350942     1.24   0.217    -.0254327     .112134
             41  |   .0705588   .0517434     1.36   0.173    -.0308565    .1719741
             42  |   .0886809   .0306116     2.90   0.004     .0286834    .1486785
             44  |   .0007096   .0356076     0.02   0.984    -.0690801    .0704992
             45  |  -.0645033   .0164698    -3.92   0.000    -.0967834   -.0322231
             46  |   .0218424   .0422534     0.52   0.605    -.0609727    .1046575
             47  |   .0544604   .0115424     4.72   0.000     .0318377     .077083
             48  |   .0526605   .0264678     1.99   0.047     .0007845    .1045365
             49  |   .0578824   .0594983     0.97   0.331    -.0587321     .174497
             50  |  -.0639811   .0278561    -2.30   0.022    -.1185781   -.0093842
             51  |   .0407227     .01273     3.20   0.001     .0157723    .0656731
             53  |   .1166724   .0514914     2.27   0.023      .015751    .2175937
             54  |   .1236941   .0113573    10.89   0.000     .1014341    .1459541
             55  |   .0813375   .0339623     2.39   0.017     .0147727    .1479024
             56  |   .0743841   .0554994     1.34   0.180    -.0343928     .183161
                 |
           _cons |   4.532124   .2917092    15.54   0.000     3.960384    5.103864
    ------------------------------------------------------------------------------
    Instrumented:  educ
    Instruments:   31.yob 32.yob 33.yob 34.yob 35.yob 36.yob 37.yob 38.yob
                   39.yob 2.pob 4.pob 5.pob 6.pob 8.pob 9.pob 10.pob 11.pob
                   12.pob 13.pob 15.pob 16.pob 17.pob 18.pob 19.pob 20.pob
                   21.pob 22.pob 23.pob 24.pob 25.pob 26.pob 27.pob 28.pob
                   29.pob 30.pob 31.pob 32.pob 33.pob 34.pob 35.pob 36.pob
                   37.pob 38.pob 39.pob 40.pob 41.pob 42.pob 44.pob 45.pob
                   46.pob 47.pob 48.pob 49.pob 50.pob 51.pob 53.pob 54.pob
                   55.pob 56.pob qob1

``` stata
* 5 使用第一季度和第二季度作为工具变量的2SLS回归
gen qob1or2 = (inlist(qob, 1, 2)) if !missing(qob)
ivregress 2sls lwklywge i.yob i.pob (educ = qob1or2), robust
```

    Instrumental variables (2SLS) regression          Number of obs   =    329,509
                                                      Wald chi2(60)   =    9695.87
                                                      Prob > chi2     =     0.0000
                                                      R-squared       =     0.0635
                                                      Root MSE        =     .65691

    ------------------------------------------------------------------------------
                 |               Robust
        lwklywge |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
    -------------+----------------------------------------------------------------
            educ |   .1217603   .0216416     5.63   0.000     .0793435    .1641771
                 |
             yob |
             31  |  -.0130185   .0061416    -2.12   0.034    -.0250557   -.0009812
             32  |  -.0221642   .0069247    -3.20   0.001    -.0357364   -.0085919
             33  |  -.0273307   .0078618    -3.48   0.001    -.0427396   -.0119219
             34  |  -.0330093   .0085206    -3.87   0.000    -.0497093   -.0163093
             35  |  -.0480764   .0096343    -4.99   0.000    -.0669592   -.0291935
             36  |  -.0500281   .0104783    -4.77   0.000    -.0705652   -.0294911
             37  |   -.058562    .011605    -5.05   0.000    -.0813075   -.0358166
             38  |  -.0637855     .01317    -4.84   0.000    -.0895983   -.0379728
             39  |  -.0766745   .0139795    -5.48   0.000    -.1040739   -.0492751
                 |
             pob |
              2  |   .1630015   .1064595     1.53   0.126    -.0456553    .3716583
              4  |    .070926   .0360786     1.97   0.049     .0002132    .1416387
              5  |   .0376879   .0124373     3.03   0.002     .0133113    .0620646
              6  |   .0569775   .0473654     1.20   0.229    -.0358569     .149812
              8  |   .0332806   .0377711     0.88   0.378    -.0407494    .1073107
              9  |   .0463463   .0369772     1.25   0.210    -.0261276    .1188202
             10  |   .0587097   .0273993     2.14   0.032      .005008    .1124113
             11  |   .0392451   .0494943     0.79   0.428    -.0577619    .1362522
             12  |  -.0508162   .0245865    -2.07   0.039     -.099005   -.0026275
             13  |  -.0078402   .0118668    -0.66   0.509    -.0310988    .0154183
             15  |   .0702381   .0545635     1.29   0.198    -.0367044    .1771805
             16  |    .005349    .043361     0.12   0.902     -.079637    .0903349
             17  |   .1163108   .0366703     3.17   0.002     .0444383    .1881832
             18  |   .0953875   .0250594     3.81   0.000      .046272     .144503
             19  |   .0214019   .0328946     0.65   0.515    -.0430703     .085874
             20  |  -.0218225   .0396818    -0.55   0.582    -.0995973    .0559524
             21  |   .1354847    .014453     9.37   0.000     .1071574     .163812
             22  |   .0750313   .0144591     5.19   0.000     .0466921    .1033705
             23  |  -.0473054   .0203238    -2.33   0.020    -.0871392   -.0074716
             24  |   .0782865   .0203096     3.85   0.000     .0384805    .1180925
             25  |   .0137008   .0395167     0.35   0.729    -.0637505    .0911522
             26  |   .1564496   .0293882     5.32   0.000     .0988497    .2140495
             27  |   .0696979   .0338125     2.06   0.039     .0034266    .1359692
             28  |  -.0088035   .0132871    -0.66   0.508    -.0348457    .0172387
             29  |   .0593924   .0238469     2.49   0.013     .0126534    .1061314
             30  |   -.012746   .0413954    -0.31   0.758    -.0938796    .0683875
             31  |   .0008809   .0382089     0.02   0.982    -.0740071    .0757689
             32  |   .0582731   .0565573     1.03   0.303    -.0525772    .1691235
             33  |   -.028857   .0270851    -1.07   0.287    -.0819428    .0242288
             34  |   .0758995   .0386892     1.96   0.050       .00007     .151729
             35  |   .0233644   .0265928     0.88   0.380    -.0287565    .0754853
             36  |   .0525008   .0440204     1.19   0.233    -.0337776    .1387792
             37  |  -.0557243   .0102633    -5.43   0.000      -.07584   -.0356086
             38  |   .0622908   .0320725     1.94   0.052    -.0005701    .1251517
             39  |   .0977027   .0282283     3.46   0.001     .0423763    .1530291
             40  |   .0205199   .0301412     0.68   0.496    -.0385558    .0795957
             41  |   .0367679   .0443901     0.83   0.408    -.0502351    .1237709
             42  |   .0686107   .0261609     2.62   0.009     .0173363    .1198851
             44  |  -.0206169   .0313881    -0.66   0.511    -.0821364    .0409026
             45  |  -.0572108   .0156303    -3.66   0.000    -.0878456    -.026576
             46  |  -.0043489   .0368797    -0.12   0.906    -.0766317     .067934
             47  |   .0574294   .0115194     4.99   0.000     .0348518    .0800069
             48  |   .0358252    .022858     1.57   0.117    -.0089756     .080626
             49  |   .0185967   .0508143     0.37   0.714    -.0809974    .1181908
             50  |  -.0762753   .0264045    -2.89   0.004    -.1280273   -.0245234
             51  |   .0448848   .0125319     3.58   0.000     .0203227    .0694469
             53  |   .0825783   .0439559     1.88   0.060    -.0035737    .1687303
             54  |    .121896   .0114593    10.64   0.000     .0994361    .1443558
             55  |   .0592543   .0291013     2.04   0.042     .0022168    .1162917
             56  |   .0412285   .0490027     0.84   0.400    -.0548151     .137272
                 |
           _cons |   4.332541   .2461557    17.60   0.000     3.850084    4.814997
    ------------------------------------------------------------------------------
    Instrumented:  educ
    Instruments:   31.yob 32.yob 33.yob 34.yob 35.yob 36.yob 37.yob 38.yob
                   39.yob 2.pob 4.pob 5.pob 6.pob 8.pob 9.pob 10.pob 11.pob
                   12.pob 13.pob 15.pob 16.pob 17.pob 18.pob 19.pob 20.pob
                   21.pob 22.pob 23.pob 24.pob 25.pob 26.pob 27.pob 28.pob
                   29.pob 30.pob 31.pob 32.pob 33.pob 34.pob 35.pob 36.pob
                   37.pob 38.pob 39.pob 40.pob 41.pob 42.pob 44.pob 45.pob
                   46.pob 47.pob 48.pob 49.pob 50.pob 51.pob 53.pob 54.pob
                   55.pob 56.pob qob1or2

``` stata
* 6 使用所有季度虚拟变量的2SLS回归
ivregress 2sls lwklywge i.yob i.pob (educ = i.qob), robust
```

    Instrumental variables (2SLS) regression          Number of obs   =    329,509
                                                      Wald chi2(60)   =    9996.12
                                                      Prob > chi2     =     0.0000
                                                      R-squared       =     0.0929
                                                      Root MSE        =     .64652

    ------------------------------------------------------------------------------
                 |               Robust
        lwklywge |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
    -------------+----------------------------------------------------------------
            educ |   .1076937   .0195571     5.51   0.000     .0693624     .146025
                 |
             yob |
             31  |  -.0110055   .0059151    -1.86   0.063    -.0225988    .0005879
             32  |  -.0191906   .0065629    -2.92   0.003    -.0320536   -.0063276
             33  |  -.0235108   .0073951    -3.18   0.001    -.0380049   -.0090167
             34  |  -.0286362    .007966    -3.59   0.000    -.0442493   -.0130231
             35  |   -.042798   .0089402    -4.79   0.000    -.0603205   -.0252754
             36  |  -.0441139   .0096716    -4.56   0.000    -.0630699   -.0251579
             37  |   -.051788   .0106655    -4.86   0.000     -.072692    -.030884
             38  |  -.0558642   .0120593    -4.63   0.000       -.0795   -.0322285
             39  |     -.0682   .0127913    -5.33   0.000    -.0932706   -.0431295
                 |
             pob |
              2  |   .1877832    .103806     1.81   0.070    -.0156729    .3912393
              4  |   .0904552   .0335368     2.70   0.007     .0247243    .1561861
              5  |    .039759    .012188     3.26   0.001     .0158709     .063647
              6  |   .0871271    .043057     2.02   0.043      .002737    .1715173
              8  |   .0559533    .034638     1.62   0.106     -.011936    .1238426
              9  |   .0690136   .0337983     2.04   0.041     .0027702    .1352569
             10  |   .0667812    .026611     2.51   0.012     .0146246    .1189379
             11  |   .0689225   .0454745     1.52   0.130    -.0202058    .1580509
             12  |   -.037266   .0228931    -1.63   0.104    -.0821357    .0076037
             13  |  -.0107995    .011547    -0.94   0.350    -.0334313    .0118322
             15  |   .0915936   .0522905     1.75   0.080    -.0108939    .1940811
             16  |   .0310043   .0398155     0.78   0.436    -.0470326    .1090413
             17  |   .1394188   .0333718     4.18   0.000     .0740114    .2048263
             18  |   .1103089   .0230406     4.79   0.000     .0651501    .1554676
             19  |   .0414928    .030153     1.38   0.169     -.017606    .1005915
             20  |   .0027997   .0362526     0.08   0.938    -.0682541    .0738535
             21  |   .1292842   .0136816     9.45   0.000     .1024687    .1560997
             22  |   .0801457   .0139029     5.76   0.000     .0528966    .1073948
             23  |  -.0381675   .0192803    -1.98   0.048    -.0759563   -.0003788
             24  |   .0885348   .0190853     4.64   0.000     .0511282    .1259414
             25  |   .0386385   .0359675     1.07   0.283    -.0318564    .1091334
             26  |   .1746004   .0268638     6.50   0.000     .1219482    .2272526
             27  |   .0904972   .0309467     2.92   0.003     .0298428    .1511516
             28  |  -.0119781   .0129262    -0.93   0.354    -.0373129    .0133568
             29  |   .0733286   .0219887     3.33   0.001     .0302315    .1164256
             30  |   .0106643   .0382853     0.28   0.781    -.0643736    .0857022
             31  |   .0240658   .0349958     0.69   0.492    -.0445248    .0926563
             32  |   .0828613   .0538912     1.54   0.124    -.0227634     .188486
             33  |  -.0163342   .0255884    -0.64   0.523    -.0664866    .0338181
             34  |   .1002406   .0352558     2.84   0.004     .0311405    .1693407
             35  |    .035644   .0250237     1.42   0.154    -.0134016    .0846895
             36  |   .0806273   .0399622     2.02   0.044     .0023028    .1589518
             37  |  -.0559031   .0101227    -5.52   0.000    -.0757433    -.036063
             38  |    .079754   .0297856     2.68   0.007     .0213752    .1381328
             39  |   .1151266   .0257716     4.47   0.000     .0646152    .1656381
             40  |   .0388023    .027609     1.41   0.160    -.0153104     .092915
             41  |   .0638269   .0406542     1.57   0.116    -.0158539    .1435078
             42  |   .0846825   .0239386     3.54   0.000     .0377637    .1316014
             44  |  -.0035391   .0292055    -0.12   0.904    -.0607808    .0537026
             45  |  -.0630505    .014983    -4.21   0.000    -.0924167   -.0336843
             46  |   .0166246   .0341154     0.49   0.626    -.0502404    .0834896
             47  |   .0550519   .0112577     4.89   0.000     .0329871    .0771166
             48  |   .0493065   .0210366     2.34   0.019     .0080757    .0905374
             49  |   .0500559    .046343     1.08   0.280    -.0407748    .1408866
             50  |  -.0664304     .02535    -2.62   0.009    -.1161155   -.0167453
             51  |   .0415519   .0121646     3.42   0.001     .0177096    .0653941
             53  |   .1098801   .0400636     2.74   0.006     .0313569    .1884034
             54  |   .1233359   .0112489    10.96   0.000     .1012884    .1453834
             55  |   .0769381   .0266644     2.89   0.004     .0246767    .1291994
             56  |   .0677788   .0455835     1.49   0.137    -.0215631    .1571208
                 |
           _cons |   4.492363   .2224326    20.20   0.000     4.056403    4.928323
    ------------------------------------------------------------------------------
    Instrumented:  educ
    Instruments:   31.yob 32.yob 33.yob 34.yob 35.yob 36.yob 37.yob 38.yob
                   39.yob 2.pob 4.pob 5.pob 6.pob 8.pob 9.pob 10.pob 11.pob
                   12.pob 13.pob 15.pob 16.pob 17.pob 18.pob 19.pob 20.pob
                   21.pob 22.pob 23.pob 24.pob 25.pob 26.pob 27.pob 28.pob
                   29.pob 30.pob 31.pob 32.pob 33.pob 34.pob 35.pob 36.pob
                   37.pob 38.pob 39.pob 40.pob 41.pob 42.pob 44.pob 45.pob
                   46.pob 47.pob 48.pob 49.pob 50.pob 51.pob 53.pob 54.pob
                   55.pob 56.pob 2.qob 3.qob 4.qob

``` stata
* 7 使用季度与出生年份交互项的2SLS回归
ivregress 2sls lwklywge i.yob i.pob (educ = i.qob#i.yob), robust
```

    Instrumental variables (2SLS) regression          Number of obs   =    329,509
                                                      Wald chi2(60)   =   10276.42
                                                      Prob > chi2     =     0.0000
                                                      R-squared       =     0.1203
                                                      Root MSE        =     .63667

    ------------------------------------------------------------------------------
                 |               Robust
        lwklywge |      Coef.   Std. Err.      z    P>|z|     [95% Conf. Interval]
    -------------+----------------------------------------------------------------
            educ |   .0869302   .0158721     5.48   0.000     .0558214     .118039
                 |
             yob |
             31  |  -.0080341   .0056077    -1.43   0.152    -.0190249    .0029568
             32  |  -.0148013   .0060249    -2.46   0.014    -.0266099   -.0029927
             33  |  -.0178723   .0066377    -2.69   0.007    -.0308819   -.0048627
             34  |  -.0221811   .0070687    -3.14   0.002    -.0360355   -.0083268
             35  |  -.0350066    .007804    -4.49   0.000    -.0503022   -.0197111
             36  |   -.035384     .00834    -4.24   0.000    -.0517301   -.0190379
             37  |   -.041789   .0090973    -4.59   0.000    -.0596193   -.0239586
             38  |  -.0441717   .0101604    -4.35   0.000    -.0640857   -.0242578
             39  |  -.0556909   .0107272    -5.19   0.000    -.0767159   -.0346659
                 |
             pob |
              2  |   .2243631   .1000944     2.24   0.025     .0281817    .4205446
              4  |    .119282   .0292686     4.08   0.000     .0619166    .1766475
              5  |   .0428161    .011893     3.60   0.000     .0195062    .0661259
              6  |   .1316306   .0353182     3.73   0.000     .0624083    .2008529
              8  |   .0894201    .029116     3.07   0.002     .0323537    .1464864
              9  |   .1024725   .0282344     3.63   0.000      .047134     .157811
             10  |   .0786955   .0255863     3.08   0.002     .0285472    .1288438
             11  |   .1127289   .0384035     2.94   0.003     .0374594    .1879984
             12  |  -.0172648   .0199603    -0.86   0.387    -.0563863    .0218567
             13  |  -.0151677   .0111901    -1.36   0.175       -.0371    .0067645
             15  |   .1231162    .048609     2.53   0.011     .0278443     .218388
             16  |   .0688739   .0338181     2.04   0.042     .0025917     .135156
             17  |   .1735284   .0275085     6.31   0.000     .1196127    .2274441
             18  |   .1323341    .019499     6.79   0.000     .0941168    .1705515
             19  |   .0711487   .0252069     2.82   0.005      .021744    .1205534
             20  |   .0391441   .0302532     1.29   0.196    -.0201511    .0984394
             21  |   .1201317   .0126328     9.51   0.000     .0953718    .1448916
             22  |    .087695   .0130515     6.72   0.000     .0621145    .1132755
             23  |  -.0246793   .0176385    -1.40   0.162      -.05925    .0098915
             24  |   .1036621   .0170321     6.09   0.000     .0702798    .1370445
             25  |   .0754487   .0296659     2.54   0.011     .0173045    .1335928
             26  |   .2013926   .0223678     9.00   0.000     .1575525    .2452327
             27  |   .1211988   .0257984     4.70   0.000     .0706349    .1717627
             28  |   -.016664   .0125145    -1.33   0.183    -.0411919    .0078639
             29  |   .0938996   .0187389     5.01   0.000      .057172    .1306271
             30  |   .0452199   .0330542     1.37   0.171    -.0195652    .1100049
             31  |   .0582887   .0294459     1.98   0.048     .0005758    .1160015
             32  |   .1191555   .0497048     2.40   0.017     .0217359    .2165752
             33  |   .0021504   .0232897     0.09   0.926    -.0434965    .0477973
             34  |   .1361701    .029067     4.68   0.000     .0791998    .1931405
             35  |   .0537697   .0225283     2.39   0.017      .009615    .0979243
             36  |   .1221445   .0327725     3.73   0.000     .0579116    .1863775
             37  |  -.0561672   .0100052    -5.61   0.000     -.075777   -.0365573
             38  |   .1055311   .0260172     4.06   0.000     .0545383    .1565239
             39  |   .1408459   .0215085     6.55   0.000     .0986899    .1830019
             40  |   .0657887   .0232463     2.83   0.005     .0202269    .1113505
             41  |   .1037684   .0340417     3.05   0.002      .037048    .1704889
             42  |   .1084059    .019958     5.43   0.000      .069289    .1475229
             44  |   .0216692   .0255476     0.85   0.396    -.0284031    .0717416
             45  |  -.0716704   .0140927    -5.09   0.000    -.0992916   -.0440492
             46  |   .0475832   .0293237     1.62   0.105    -.0098901    .1050565
             47  |   .0515424   .0109628     4.70   0.000     .0300557    .0730291
             48  |   .0692062    .017847     3.88   0.000     .0342268    .1041857
             49  |   .0964925   .0385202     2.50   0.012     .0209942    .1719907
             50  |  -.0518984   .0238097    -2.18   0.029    -.0985646   -.0052322
             51  |   .0366321   .0117198     3.13   0.002     .0136617    .0596025
             53  |     .15018   .0332903     4.51   0.000     .0849321    .2154278
             54  |   .1254613   .0109958    11.41   0.000     .1039099    .1470128
             55  |    .103041   .0223912     4.60   0.000      .059155     .146927
             56  |   .1069695   .0398744     2.68   0.007     .0288172    .1851218
                 |
           _cons |   4.728274   .1806406    26.18   0.000     4.374225    5.082324
    ------------------------------------------------------------------------------
    Instrumented:  educ
    Instruments:   31.yob 32.yob 33.yob 34.yob 35.yob 36.yob 37.yob 38.yob
                   39.yob 2.pob 4.pob 5.pob 6.pob 8.pob 9.pob 10.pob 11.pob
                   12.pob 13.pob 15.pob 16.pob 17.pob 18.pob 19.pob 20.pob
                   21.pob 22.pob 23.pob 24.pob 25.pob 26.pob 27.pob 28.pob
                   29.pob 30.pob 31.pob 32.pob 33.pob 34.pob 35.pob 36.pob
                   37.pob 38.pob 39.pob 40.pob 41.pob 42.pob 44.pob 45.pob
                   46.pob 47.pob 48.pob 49.pob 50.pob 51.pob 53.pob 54.pob
                   55.pob 56.pob 2.qob#30b.yob 2.qob#31.yob 2.qob#32.yob
                   2.qob#33.yob 2.qob#34.yob 2.qob#35.yob 2.qob#36.yob
                   2.qob#37.yob 2.qob#38.yob 2.qob#39.yob 3.qob#30b.yob
                   3.qob#31.yob 3.qob#32.yob 3.qob#33.yob 3.qob#34.yob
                   3.qob#35.yob 3.qob#36.yob 3.qob#37.yob 3.qob#38.yob
                   3.qob#39.yob 4.qob#30b.yob 4.qob#31.yob 4.qob#32.yob
                   4.qob#33.yob 4.qob#34.yob 4.qob#35.yob 4.qob#36.yob
                   4.qob#37.yob 4.qob#38.yob 4.qob#39.yob